In [1]:
import pysmile
import pysmile_license
import numpy as np
import pandas as pd

from network_functions import calculate_network_utilities, new_screening_strategy, old_screening_strategy, create_folders_logger
from simulations import plot_classification_results
from plots import plot_estimations_w_error_bars, plot_screening_counts

from preprocessing import preprocessing
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
import matplotlib.pyplot as plt

import yaml
with open('config.yaml', 'r') as file:
    cfg = yaml.safe_load(file)

import argparse
import pdb
import matplotlib
matplotlib.use('Agg')

import logging
import datetime 
import os

In [2]:
# from main import main
from use_case_new_strategy import use_case_new_strategy
from full_example import full_example
from update import update_influence_diagram
from network_functions import create_folders_logger
from sens_analysis_elicitation import sens_analysis_elicitation
from sens_analysis_PE_method import sens_analysis_PE_method

### Preferences model

In [4]:
cfg["patient_chars"]

{'Age': 'age_4_adult',
 'Sex': 'M',
 'SD': 'SD_2_normal',
 'PA': 'PA_2',
 'Smoking': 'sm_1_not_smoker',
 'BMI': 'bmi_2_normal',
 'Alcohol': 'low'}

## Use cases

### Individual cases

**Benchmark patient**

In [5]:
label = 'notebook_benchmark_patient'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = cfg["patient_chars"],
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label
)

2024-11-21 10:40:02,146 - INFO - Model type: linear
2024-11-21 10:40:02,146 - INFO - Reading network...
2024-11-21 10:40:02,162 - INFO - Calculating information values...
2024-11-21 10:40:53,041 - INFO - Eliciting value of comfort...
2024-11-21 10:40:53,042 - INFO - No elicitation of lambda values, taking default values...
2024-11-21 10:40:53,043 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2024-11-21 10:40:53,044 - INFO - Saving network...
2024-11-21 10:40:53,076 - INFO - Plotting info functions...
2024-11-21 10:41:26,012 - INFO - Calculating final utilities...
2024-11-21 10:41:26,019 - INFO - Best info: 15.299634978175163
2024-11-21 10:41:26,021 - INFO - Worst info: 4.450147717014403e-308
2024-11-21 10:41:26,021 - INFO - PE info: 4
2024-11-21 10:41:26,022 - INFO - v_best: 104.06788025218766
2024-11-21 10:41:26,022 - INFO - v_worst: -3.91023528629925

**Benchmark patient risk seeking**

In [4]:
label = 'notebook_benchmark_risk_seeking'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = cfg["patient_chars"],
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label,
    change_risk_param=True,
    rho_param= 0.005
)

2024-11-20 17:16:51,640 - INFO - Model type: linear
2024-11-20 17:16:51,640 - INFO - Model type: linear
2024-11-20 17:16:51,642 - INFO - Reading network...
2024-11-20 17:16:51,642 - INFO - Reading network...
2024-11-20 17:16:51,658 - INFO - Calculating information values...
2024-11-20 17:16:51,658 - INFO - Calculating information values...
2024-11-20 17:17:43,696 - INFO - Eliciting value of comfort...
2024-11-20 17:17:43,696 - INFO - Eliciting value of comfort...
2024-11-20 17:17:43,698 - INFO - No elicitation of lambda values, taking default values...
2024-11-20 17:17:43,698 - INFO - No elicitation of lambda values, taking default values...
2024-11-20 17:17:43,699 - INFO - Adding noise to the lambda values...
2024-11-20 17:17:43,699 - INFO - Adding noise to the lambda values...
2024-11-20 17:17:43,700 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2024

**Benchmark patient added evidence**

In [ ]:
label = 'notebook_benchmark_addded_ev'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

patient_chars = cfg["patient_chars"].copy()
patient_chars["Diabetes"] = True
patient_chars["Hypertension"] = True

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = patient_chars,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label,
)

2024-11-21 10:36:17,431 - INFO - Model type: linear
2024-11-21 10:36:17,431 - INFO - Model type: linear
2024-11-21 10:36:17,438 - INFO - Reading network...
2024-11-21 10:36:17,438 - INFO - Reading network...
2024-11-21 10:36:17,452 - INFO - Calculating information values...
2024-11-21 10:36:17,452 - INFO - Calculating information values...
2024-11-21 10:37:07,971 - INFO - Eliciting value of comfort...
2024-11-21 10:37:07,971 - INFO - Eliciting value of comfort...
2024-11-21 10:37:07,973 - INFO - No elicitation of lambda values, taking default values...
2024-11-21 10:37:07,973 - INFO - No elicitation of lambda values, taking default values...
2024-11-21 10:37:07,974 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2024-11-21 10:37:07,974 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.8019

**Exogenous variable p = 0.02**

In [6]:
label = 'notebook_exogenous_var_002'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

exog_char ={"Age": "age_2_young", 
    "Sex": "M",
    "SES": "ses_0",
    "SD": "SD_1_short",
    "PA": "PA_1",
    "Depression": False,
    "Smoking": "sm_1_not_smoker",
    "Anxiety": False,
    "BMI": "bmi_1_underweight",
    "Alcohol": "high",
    "Diabetes": False,
    "Hypertension": False,
    "Hyperchol_": False,
    }


update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = exog_char,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label,
    exogenous_var_prob= 0.02
)

2024-11-20 17:19:47,261 - INFO - Model type: linear
2024-11-20 17:19:47,261 - INFO - Reading network...
2024-11-20 17:19:47,278 - INFO - Calculating information values...
2024-11-20 17:20:42,898 - INFO - Eliciting value of comfort...
2024-11-20 17:20:42,899 - INFO - No elicitation of lambda values, taking default values...
2024-11-20 17:20:42,901 - INFO - Adding noise to the lambda values...
2024-11-20 17:20:42,902 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2024-11-20 17:20:42,902 - INFO - Saving network...
2024-11-20 17:20:42,941 - INFO - Plotting info functions...
2024-11-20 17:21:17,239 - INFO - Calculating final utilities...
2024-11-20 17:21:17,245 - INFO - Best info: 15.299634978175163
2024-11-20 17:21:17,246 - INFO - Worst info: 4.450147717014403e-308
2024-11-20 17:21:17,246 - INFO - PE info: 4
2024-11-20 17:21:17,247 - INFO - v_best: 104.0678

**Exogenous variable p = 0.1**

In [7]:
label = 'notebook_exogenous_var_01'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

exog_char ={"Age": "age_2_young", 
    "Sex": "M",
    "SES": "ses_0",
    "SD": "SD_1_short",
    "PA": "PA_1",
    "Depression": False,
    "Smoking": "sm_1_not_smoker",
    "Anxiety": False,
    "BMI": "bmi_1_underweight",
    "Alcohol": "high",
    "Diabetes": False,
    "Hypertension": False,
    "Hyperchol_": False,
    }

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = exog_char,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label,
    exogenous_var_prob= 0.1
)

2024-11-20 17:21:17,401 - INFO - Model type: linear
2024-11-20 17:21:17,402 - INFO - Reading network...
2024-11-20 17:21:17,422 - INFO - Calculating information values...
2024-11-20 17:22:08,424 - INFO - Eliciting value of comfort...
2024-11-20 17:22:08,425 - INFO - No elicitation of lambda values, taking default values...
2024-11-20 17:22:08,425 - INFO - Adding noise to the lambda values...
2024-11-20 17:22:08,427 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2024-11-20 17:22:08,428 - INFO - Saving network...
2024-11-20 17:22:08,465 - INFO - Plotting info functions...
2024-11-20 17:22:42,209 - INFO - Calculating final utilities...
2024-11-20 17:22:42,216 - INFO - Best info: 15.299634978175163
2024-11-20 17:22:42,217 - INFO - Worst info: 4.450147717014403e-308
2024-11-20 17:22:42,217 - INFO - PE info: 4
2024-11-20 17:22:42,218 - INFO - v_best: 104.0678

### Assessing the French strategy

**Patient 1**

In [8]:
label = 'notebook_4_2_patient_1'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

patient_chars = {
    "Age": "age_5_old_adult", 
    "Sex": "M",
    "SD": "SD_2_normal",
    "PA": "PA_2",
    "Smoking": "sm_1_not_smoker",
    "BMI": "bmi_2_normal",
    "Alcohol": "low",
    "Diabetes": False,
    "Hypertension": False,
}

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = patient_chars,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label
)

2024-11-20 17:22:42,397 - INFO - Model type: linear
2024-11-20 17:22:42,398 - INFO - Reading network...
2024-11-20 17:22:42,414 - INFO - Calculating information values...
2024-11-20 17:23:37,585 - INFO - Eliciting value of comfort...
2024-11-20 17:23:37,586 - INFO - No elicitation of lambda values, taking default values...
2024-11-20 17:23:37,586 - INFO - Adding noise to the lambda values...
2024-11-20 17:23:37,587 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2024-11-20 17:23:37,588 - INFO - Saving network...
2024-11-20 17:23:37,634 - INFO - Plotting info functions...
2024-11-20 17:24:13,691 - INFO - Calculating final utilities...
2024-11-20 17:24:13,697 - INFO - Best info: 15.299634978175163
2024-11-20 17:24:13,698 - INFO - Worst info: 4.450147717014403e-308
2024-11-20 17:24:13,699 - INFO - PE info: 4
2024-11-20 17:24:13,699 - INFO - v_best: 104.0678

**Patient 2**

In [9]:
label = 'notebook_4_2_patient_2'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

patient_chars = {
    "Age": "age_4_adult", 
    "Sex": "M",
    "SD": "SD_2_normal",
    "PA": "PA_2",
    "Smoking": "sm_1_not_smoker",
    "BMI": "bmi_2_normal",
    "Alcohol": "low",
    "Diabetes": True,
    "Hypertension": True,
}

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = patient_chars,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label
)

2024-11-20 17:24:13,911 - INFO - Model type: linear
2024-11-20 17:24:13,912 - INFO - Reading network...
2024-11-20 17:24:13,935 - INFO - Calculating information values...
2024-11-20 17:25:10,072 - INFO - Eliciting value of comfort...
2024-11-20 17:25:10,073 - INFO - No elicitation of lambda values, taking default values...
2024-11-20 17:25:10,073 - INFO - Adding noise to the lambda values...
2024-11-20 17:25:10,075 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2024-11-20 17:25:10,075 - INFO - Saving network...
2024-11-20 17:25:10,126 - INFO - Plotting info functions...
2024-11-20 17:25:46,524 - INFO - Calculating final utilities...
2024-11-20 17:25:46,531 - INFO - Best info: 15.299634978175163
2024-11-20 17:25:46,531 - INFO - Worst info: 4.450147717014403e-308
2024-11-20 17:25:46,532 - INFO - PE info: 4
2024-11-20 17:25:46,533 - INFO - v_best: 104.0678

**Patient 3**

In [10]:
label = 'notebook_4_2_patient_3'
logger, log_dir = create_folders_logger(label = label)

patient_chars = {
    "Age": "age_4_adult", 
    "Sex": "M",
    "SD": "SD_2_normal",
    "PA": "PA_1",
    "Smoking": "sm_3_ex_smoker",
    "BMI": "bmi_3_overweight",
    "Alcohol": "high",
}

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = patient_chars,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label
)

2024-11-20 17:25:46,660 - INFO - Model type: linear
2024-11-20 17:25:46,661 - INFO - Reading network...
2024-11-20 17:25:46,676 - INFO - Calculating information values...
2024-11-20 17:26:42,742 - INFO - Eliciting value of comfort...
2024-11-20 17:26:42,743 - INFO - No elicitation of lambda values, taking default values...
2024-11-20 17:26:42,744 - INFO - Adding noise to the lambda values...
2024-11-20 17:26:42,745 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2024-11-20 17:26:42,746 - INFO - Saving network...
2024-11-20 17:26:42,787 - INFO - Plotting info functions...
2024-11-20 17:27:20,877 - INFO - Calculating final utilities...
2024-11-20 17:27:20,884 - INFO - Best info: 15.299634978175163
2024-11-20 17:27:20,885 - INFO - Worst info: 4.450147717014403e-308
2024-11-20 17:27:20,886 - INFO - PE info: 4
2024-11-20 17:27:20,886 - INFO - v_best: 104.0678

### Designing a national personalised strategy

In [ ]:
label = 'notebook_personalised_screening_str'
logger, log_dir = create_folders_logger(label=label, output_dir = 'notebook/' + label, date = False, time = False)

use_case_new_strategy(
    logger = logger,
    single_run=True,
    output_dir = 'notebook/' + label,
    log_dir = log_dir,
    run_label = ''
)

2024-11-20 17:27:21,303 - INFO - Configuration variables of interest:
2024-11-20 17:27:21,305 - INFO - Single run: True
2024-11-20 17:27:21,305 - INFO - Use all variables: True
2024-11-20 17:27:21,306 - INFO - Use case with new test: False
2024-11-20 17:27:21,307 - INFO - PE method: {'PE_cost': 5, 'PE_info': 4, 'PE_prob': 0.7}
2024-11-20 17:27:21,307 - INFO - Reading the network file...
2024-11-20 17:27:21,329 - INFO - Located at: outputs/linear_rel_point_cond_mut_info_elicitFalse_newtestFalse/decision_models/DM_screening_rel_point_cond_mut_info_linear.xdsl
2024-11-20 17:27:21,331 - INFO - Comfort values: 1 - 4.011942483017116, 2 - 4.168480067521723, 3 - 6.801984517973132, 4 - 7.0
2024-11-20 17:27:21,978 - INFO - All variables are kept in the dataframe for calculation of utilities.
2024-11-20 17:27:21,979 - INFO - Operational limits for the screening strategies: {'No_scr_no_col': inf, 'No_scr_col': 3000, 'gFOBT': 30000, 'FIT': 42000, 'Blood_based': 7000, 'Stool_DNA': 5000, 'CTC': 2000,

### Benchmarking of new screening devices

In [6]:
cfg["patient_chars"]

{'Age': 'age_4_adult',
 'Sex': 'M',
 'SD': 'SD_2_normal',
 'PA': 'PA_2',
 'Smoking': 'sm_1_not_smoker',
 'BMI': 'bmi_2_normal',
 'Alcohol': 'low'}

In [7]:
label = 'notebook_benchmark_new_screening'
logger, log_dir = create_folders_logger(label=label, output_dir = 'notebook/' + label, date = False, time = False)

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = cfg["patient_chars"],
    new_test = True,
    logger = logger,
    output_dir = 'notebook/' + label
)

2024-11-21 10:41:32,030 - INFO - Model type: linear
2024-11-21 10:41:32,030 - INFO - Model type: linear
2024-11-21 10:41:32,032 - INFO - Reading network...
2024-11-21 10:41:32,032 - INFO - Reading network...
2024-11-21 10:41:32,046 - INFO - Calculating information values...
2024-11-21 10:41:32,046 - INFO - Calculating information values...
2024-11-21 10:42:23,638 - INFO - Eliciting value of comfort...
2024-11-21 10:42:23,638 - INFO - Eliciting value of comfort...
2024-11-21 10:42:23,640 - INFO - No elicitation of lambda values, taking default values...
2024-11-21 10:42:23,640 - INFO - No elicitation of lambda values, taking default values...
2024-11-21 10:42:23,641 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2024-11-21 10:42:23,641 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.8019

In [9]:
cfg["lambda_list"]

[4.011942483017116, 4.168480067521723, 6.801984517973132, 7.0]

In [ ]:
label = 'notebook_new_test_screening_strategy'
logger, log_dir = create_folders_logger(label=label, output_dir = 'notebook/' + label, date = False, time = False)

use_case_new_strategy(
    use_case_new_test = True,
    single_run=True,
    output_dir = 'notebook/' + label,
    log_dir = log_dir,
    run_label = '',
)

2024-11-21 10:59:42,909 - INFO - Configuration variables of interest:
2024-11-21 10:59:42,909 - INFO - Configuration variables of interest:
2024-11-21 10:59:42,909 - INFO - Configuration variables of interest:
2024-11-21 10:59:42,909 - INFO - Configuration variables of interest:
2024-11-21 10:59:42,911 - INFO - Single run: True
2024-11-21 10:59:42,911 - INFO - Single run: True
2024-11-21 10:59:42,911 - INFO - Single run: True
2024-11-21 10:59:42,911 - INFO - Single run: True
2024-11-21 10:59:42,913 - INFO - Use all variables: True
2024-11-21 10:59:42,913 - INFO - Use all variables: True
2024-11-21 10:59:42,913 - INFO - Use all variables: True
2024-11-21 10:59:42,913 - INFO - Use all variables: True
2024-11-21 10:59:42,914 - INFO - Use case with new test: True
2024-11-21 10:59:42,914 - INFO - Use case with new test: True
2024-11-21 10:59:42,914 - INFO - Use case with new test: True
2024-11-21 10:59:42,914 - INFO - Use case with new test: True
2024-11-21 10:59:42,916 - INFO - PE method: 

## Sensitivity analysis

### Sensitivity analysis for the PE

In [3]:
label = 'notebook_sens_analysis_PE_method'
logger, log_dir = create_folders_logger(label=label, output_dir = 'notebook/' + label, date = False, time = False)

sens_analysis_PE_method(
    label = label,
    output_dir = 'notebook/' + label,
    log_dir = log_dir,  
)

2024-11-20 13:30:35,094 - INFO - PE_info_array: [3.8 4.  4.2]
2024-11-20 13:30:35,094 - INFO - PE_info_array: [3.8 4.  4.2]
2024-11-20 13:30:35,096 - INFO - PE_cost_array: [ 5 10 50]
2024-11-20 13:30:35,096 - INFO - PE_cost_array: [ 5 10 50]


2024-11-20 13:30:38,737 - INFO - Params: [1.0105405558122231, 0.8618081053948691, 0.04071571451462677]
2024-11-20 13:30:38,737 - INFO - Params: [1.0105405558122231, 0.8618081053948691, 0.04071571451462677]
2024-11-20 13:31:38,305 - INFO - PE_info: 3.8, PE_cost: 5, DONE!
2024-11-20 13:31:38,305 - INFO - PE_info: 3.8, PE_cost: 5, DONE!
2024-11-20 13:31:39,096 - INFO - Params: [1.0170169520480368, 0.8677488153052746, 0.040592639410855884]
2024-11-20 13:31:39,096 - INFO - Params: [1.0170169520480368, 0.8677488153052746, 0.040592639410855884]
2024-11-20 13:32:37,544 - INFO - PE_info: 3.8, PE_cost: 10, DONE!
2024-11-20 13:32:37,544 - INFO - PE_info: 3.8, PE_cost: 10, DONE!
2024-11-20 13:32:38,164 - INFO - Params: [1.0151957194469061, 0.8624584291169836, 0.04169819311246861]
2024-11-20 13:32:38,164 - INFO - Params: [1.0151957194469061, 0.8624584291169836, 0.04169819311246861]
2024-11-20 13:33:37,272 - INFO - PE_info: 3.8, PE_cost: 50, DONE!
2024-11-20 13:33:37,272 - INFO - PE_info: 3.8, PE_co

### Sensitivity analysis for comfort parameters

In [4]:
label = 'notebook_sens_analysis_comf_param'
logger, log_dir = create_folders_logger(label = label, output_dir= 'notebook/' + label, date = False, time = False)

sens_analysis_elicitation(
    label = label,
    output_dir = 'notebook/' + label
)

2024-11-20 13:39:39,037 - INFO - Configuration variables of interest:
2024-11-20 13:39:39,037 - INFO - Configuration variables of interest:
2024-11-20 13:39:39,037 - INFO - Configuration variables of interest:
2024-11-20 13:39:39,037 - INFO - Configuration variables of interest:
2024-11-20 13:39:39,039 - INFO - Single run: True
2024-11-20 13:39:39,039 - INFO - Single run: True
2024-11-20 13:39:39,039 - INFO - Single run: True
2024-11-20 13:39:39,039 - INFO - Single run: True
2024-11-20 13:39:39,040 - INFO - Use all variables: True
2024-11-20 13:39:39,040 - INFO - Use all variables: True
2024-11-20 13:39:39,040 - INFO - Use all variables: True
2024-11-20 13:39:39,040 - INFO - Use all variables: True
2024-11-20 13:39:39,042 - INFO - Use case with new test: False
2024-11-20 13:39:39,042 - INFO - Use case with new test: False
2024-11-20 13:39:39,042 - INFO - Use case with new test: False
2024-11-20 13:39:39,042 - INFO - Use case with new test: False
2024-11-20 13:39:39,043 - INFO - PE meth

Counts: best_option
No_scr_no_col    288682
No_scr_col            0
gFOBT                 0
FIT                3201
Blood_based           0
Stool_DNA         47824
CTC                   0
Colon_capsule         0
Name: count, dtype: int64


In [ ]:
label = 'notebook_sens_analysis_comf_param'
logger, log_dir = create_folders_logger(label = label, output_dir= 'notebook/' + label, date = False, time = False)

lambda_list = cfg["lambda_list"].copy()
lambda_list[2] = 6.3

sens_analysis_elicitation(
    lambda_list = lambda_list,
    label = label,
    output_dir = 'notebook/' + label
)

2024-11-20 13:42:02,647 - INFO - Configuration variables of interest:
2024-11-20 13:42:02,647 - INFO - Configuration variables of interest:
2024-11-20 13:42:02,647 - INFO - Configuration variables of interest:
2024-11-20 13:42:02,649 - INFO - Single run: True
2024-11-20 13:42:02,649 - INFO - Single run: True
2024-11-20 13:42:02,649 - INFO - Single run: True
2024-11-20 13:42:02,650 - INFO - Use all variables: True
2024-11-20 13:42:02,650 - INFO - Use all variables: True
2024-11-20 13:42:02,650 - INFO - Use all variables: True
2024-11-20 13:42:02,651 - INFO - Use case with new test: False
2024-11-20 13:42:02,651 - INFO - Use case with new test: False
2024-11-20 13:42:02,651 - INFO - Use case with new test: False
2024-11-20 13:42:02,652 - INFO - PE method: {'PE_cost': 10, 'PE_info': 4, 'PE_prob': 0.7}
2024-11-20 13:42:02,652 - INFO - PE method: {'PE_cost': 10, 'PE_info': 4, 'PE_prob': 0.7}
2024-11-20 13:42:02,652 - INFO - PE method: {'PE_cost': 10, 'PE_info': 4, 'PE_prob': 0.7}
2024-11-20

Counts: best_option
No_scr_no_col    313960
No_scr_col            0
gFOBT                 0
FIT                   0
Blood_based           0
Stool_DNA         25747
CTC                   0
Colon_capsule         0
Name: count, dtype: int64


In [6]:
label = 'notebook_sens_analysis_comf_param'
logger, log_dir = create_folders_logger(label = label, output_dir= 'notebook/' + label, date = False, time = False)

lambda_list = cfg["lambda_list"]
lambda_list[0] = 4.8
lambda_list[1] = 5

sens_analysis_elicitation(
    lambda_list = lambda_list,
    label = label,
    output_dir = 'notebook/' + label
)

2024-11-20 13:44:27,562 - INFO - Configuration variables of interest:
2024-11-20 13:44:27,562 - INFO - Configuration variables of interest:
2024-11-20 13:44:27,564 - INFO - Single run: True
2024-11-20 13:44:27,564 - INFO - Single run: True
2024-11-20 13:44:27,565 - INFO - Use all variables: True
2024-11-20 13:44:27,565 - INFO - Use all variables: True
2024-11-20 13:44:27,566 - INFO - Use case with new test: False
2024-11-20 13:44:27,566 - INFO - Use case with new test: False
2024-11-20 13:44:27,567 - INFO - PE method: {'PE_cost': 10, 'PE_info': 4, 'PE_prob': 0.7}
2024-11-20 13:44:27,567 - INFO - PE method: {'PE_cost': 10, 'PE_info': 4, 'PE_prob': 0.7}
2024-11-20 13:44:27,568 - INFO - Change lambdas? True
2024-11-20 13:44:27,568 - INFO - Change lambdas? True
2024-11-20 13:44:27,569 - INFO - Read lambda list from config? True
2024-11-20 13:44:27,569 - INFO - Read lambda list from config? True
2024-11-20 13:44:27,570 - INFO - Lambdas: [4.8, 5, 6.3, 7.0]
2024-11-20 13:44:27,570 - INFO - La

Counts: best_option
No_scr_no_col    209634
No_scr_col            0
gFOBT                 0
FIT                   0
Blood_based           0
Stool_DNA        130073
CTC                   0
Colon_capsule         0
Name: count, dtype: int64
